In [ ]:
#This code is submitted for smartathon for Visual pollution detection using VGG-16 model with transfer learning approach

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import required libraries
import numpy as np
import pandas as pd
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
# Load the annotations from the CSV file
annotations = pd.read_csv("/content/drive/MyDrive/dataset/sample_train.csv")

# Extract the classes and file paths from the annotations
classes = annotations["name"].values
file_paths = annotations["image_path"].values

# Extract the bounding boxes from the annotations
bounding_boxes = annotations[["xmin", "ymin", "xmax", "ymax"]].values


In [ ]:
# Load the images and their corresponding classes and bounding boxes
X = []
y = []
bboxes = []
for file_path, class_name, bbox in zip(file_paths, classes, bounding_boxes):
    img = Image.open("/content/drive/MyDrive/dataset/sample_images/" + file_path)
    img = img.resize((960, 960))
    X.append(np.array(img))
    y.append(class_name)
    bboxes.append(bbox)

# Convert the classes and bounding boxes to numpy arrays
num_classes = len(np.unique(y))
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = to_categorical(y, num_classes)
bboxes = np.array(bboxes)
print(num_classes)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val, bboxes_train, bboxes_val = train_test_split(X, y, bboxes, test_size=1)



11


In [ ]:
# Load the VGG16 model with pre-trained weights
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(960, 960, 3))

In [ ]:
# Add a new fully connected layer for the 11 classes
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(11, activation='softmax')(x)


In [ ]:
# Create a new model with the new fully connected layer
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
# Compile the new model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
#labelling as binary
label_as_binary = LabelBinarizer()
train_y = label_as_binary.fit_transform(y_train)
print(np.unique(y_train))
print(y_train.shape)

[0. 1.]
(202, 11)


In [ ]:
# Train the model
model.fit(np.array(X_train),y_train, batch_size=5, epochs=10, validation_data=(np.array(X_val), np.array(y_val)))


Epoch 1/10
41/41 [==============================] - 56s 965ms/step - loss: 628.5138 - accuracy: 0.4356 - val_loss: 1972.6261 - val_accuracy: 0.0000e+00
Epoch 2/10
41/41 [==============================] - 30s 726ms/step - loss: 388.2005 - accuracy: 0.7822 - val_loss: 1225.9535 - val_accuracy: 0.0000e+00
Epoch 3/10
41/41 [==============================] - 30s 724ms/step - loss: 258.5515 - accuracy: 0.8267 - val_loss: 2230.8132 - val_accuracy: 0.0000e+00
Epoch 4/10
41/41 [==============================] - 30s 736ms/step - loss: 290.8881 - accuracy: 0.8218 - val_loss: 1807.8984 - val_accuracy: 0.0000e+00
Epoch 5/10
41/41 [==============================] - 30s 728ms/step - loss: 277.1537 - accuracy: 0.8515 - val_loss: 1771.5693 - val_accuracy: 0.0000e+00
Epoch 6/10
41/41 [==============================] - 30s 727ms/step - loss: 384.4093 - accuracy: 0.8069 - val_loss: 1918.4377 - val_accuracy: 0.0000e+00
Epoch 7/10
41/41 [==============================] - 30s 731ms/step - loss: 324.2781 - ac

In [ ]:
# Save the model
model.save("vgg16_transfer_learning_model_1.h5")

In [ ]:
#model prediction

# Extract the file paths from the annotations

annots = pd.read_csv("/content/drive/MyDrive/dataset/sample_train.csv"

file_paths = annots["image_path"].values

#for storing in CSV file
results = pd.DataFrame(columns=["class","image_path","name","xmax","xmin","ymax","ymin"])

# Load the test image

for file_path in file_paths:
    img = Image.open("/content/drive/MyDrive/dataset/sample_images/" + file_path)
    img = img.resize((960, 960))
    x = np.asarray(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    # Make predictions
    preds = model.predict(x)
    # Get the class with the highest probability
    predicted_class = np.argmax(preds)
    # Get the class label
    class_label = classes[predicted_class]
    #bounding box metrics
    bbox = bounding_boxes[predicted_class]
    # Get the bounding box coordinates
    xmin, ymin, xmax, ymax = bbox

    
    # Append the results to the dataframe
    results = results.append({"class": predicted_class,"image_path":file_path, "name": class_label, "xmax": xmax,"xmin": xmin,"ymax": ymax,"ymin": ymin}, ignore_index=True)
    results.to_csv("/content/drive/MyDrive/dataset/result.csv", index=False)

    

